In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as  np
import matplotlib.pyplot as  plt
import seaborn as sns
pd.set_option('display.max_columns',None)
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
df= pd.read_csv(r'/kaggle/input/heart-attack-analysis-prediction-dataset/heart.csv',header=0)
df.head()

In [ ]:
# understanding the data 
print(df.shape)
print(df.dtypes)

print(df.info())


In [ ]:
df.columns

In [ ]:
df.describe()

In [ ]:
#Understanding the categorical variables
a=['sex','cp','fbs','restecg','exng','slp','caa','thall']
for i in a:
    print(df[i].value_counts())
    sns.countplot(df[i])
    plt.title([i])
    plt.show()


# Handling Missing  Values

In [ ]:
# as per the data set  description the variable 'Thal  has to have  only  3  values 1,2 & 3 .  the  value  0  is  given  are  null- values.
df.thall=df.thall.replace(0,np.nan)

print(df.thall.mode())

In [ ]:
df.thall.fillna(df.thall.mode()[0],inplace=True)
print(df.thall.value_counts())


In [ ]:
# checking for  other  mising values 
df.isnull().sum().sort_values(ascending=False)

In [ ]:
# Checking for correlation between variables 
X=df[["age","trtbps","chol","thalachh","oldpeak"]] 
corr_df=X.corr(method="pearson")
print(corr_df)

sns.heatmap(corr_df,vmax=1.0,vmin=-1.0,annot=True)
plt.savefig("heatmap.jpg")

In [ ]:
# Relationship between Sex, age and  output 
sns.barplot(x='sex',y='age',data=df,hue='output',palette='rocket')
plt.show()

In [ ]:
# Different chestpains  and coletrol and  output
sns.barplot(x='cp',y='chol',data=df,hue='output')
plt.show()

In [ ]:
# check biasness in data 
sns.countplot(df.output)
plt.show()

#Insight- Data is  not biased 

In [ ]:

sns.pairplot(x_vars=["age","trtbps","chol","thalachh","oldpeak"],y_vars='age',data=df,hue='output')
plt.show()


# Splitting the Variables

In [ ]:
X=df.values[:,0:-1]
Y=df.values[:,-1]
Y=Y.astype(int)

In [ ]:
print(X.shape)
print(Y.shape)

# Preprocessing steps

In [ ]:
# Scaling the data
from sklearn.preprocessing  import StandardScaler
scaler=StandardScaler()
scaler.fit(X)
X=scaler.transform(X)
print(X)

# Splitting the data

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.3,random_state=10)

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
lm=LogisticRegression()
lm.fit(X_train,Y_train)
Y_predict=lm.predict(X_test)

In [ ]:
print(list(zip(Y_test,Y_predict)))

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

cfm=confusion_matrix(Y_test,Y_predict) 
print(cfm)

print("Classification report: ")

print(classification_report(Y_test,Y_predict))

acc=accuracy_score(Y_test, Y_predict)
print("Accuracy of the model: ",acc)

# Tuning Logistic Regression 

In [ ]:
y_pred_prob = lm.predict_proba(X_test)
print(y_pred_prob)

#the accuracy has  slightly increased  to  

In [ ]:
for i in np.arange(0.4,0.61,0.01):
    predict_new=np.where(y_pred_prob[:,1]>i,1,0)
    cfm=confusion_matrix(Y_test,predict_new)
    total_err=cfm[0,1]+cfm[1,0]
    print('errors at threshold:',i,':',total_err,'Type_1:',cfm[0,1],'type2_error:',cfm[1,0])

#As its a sensitive data set and we want  to keep the type 2 error  minimal, we choose  0.4  

In [ ]:
y_pred_class=[]
for i in y_pred_prob[:,1]:
    if i>0.40:
        y_pred_class.append(1)
    else:
        y_pred_class.append(0)
print(y_pred_class)

In [ ]:
from sklearn.metrics  import confusion_matrix, accuracy_score,classification_report
accuracy_score= accuracy_score(Y_test,y_pred_class)
print('accuracy:',accuracy_score)
cfm=confusion_matrix(Y_test,y_pred_class)
print('confusion_matrix',cfm)
print(classification_report(Y_test,y_pred_class))

In [ ]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
estimators=[]
svc_model=SVC(kernel='rbf',C=30.0,gamma=0.01)
estimators.append(('SVM',svc_model))
model_RF=RandomForestClassifier(n_estimators=35,random_state=10,max_depth=10,min_samples_leaf=5,min_samples_split=6)
estimators.append(('RF',model_RF))
model_LR=LogisticRegression()
estimators.append(('LR',model_LR))
model_knn=model_knn=KNeighborsClassifier(n_neighbors=int(np.sqrt(len(X_train))),metric='manhattan')
estimators.append(('KNN',model_knn))

In [ ]:
ensemble=VotingClassifier(estimators)
ensemble.fit(X_train,Y_train)
Y_pred=ensemble.predict(X_test)

In [ ]:
print(list(zip(Y_test,Y_pred)))

In [ ]:
from sklearn.metrics  import confusion_matrix, accuracy_score,classification_report
accuracy_score= accuracy_score(Y_test,Y_pred)
print('accuracy:',accuracy_score)
cfm=confusion_matrix(Y_test,Y_pred)
print('confusion_matrix',cfm)
print(classification_report(Y_test,Y_pred))

# USING THE  MODELS  GIVING  BETTER RECALL VALUE  FOR  CLASS1 

In [ ]:
estimators=[]
svc_model=SVC(kernel='rbf',C=30.0,gamma=0.01)
estimators.append(('SVM',svc_model))
model_RF=RandomForestClassifier(n_estimators=35,random_state=10,max_depth=10,min_samples_leaf=5,min_samples_split=6)
estimators.append(('RF',model_RF))


In [ ]:
ensemble=VotingClassifier(estimators)
ensemble.fit(X_train,Y_train)
Y_pred=ensemble.predict(X_test)

In [ ]:
print(list(zip(Y_test,Y_pred)))

In [ ]:
from sklearn.metrics  import confusion_matrix, accuracy_score,classification_report
accuracy_score= accuracy_score(Y_test,Y_pred)
print('accuracy:',accuracy_score)
cfm=confusion_matrix(Y_test,Y_pred)
print('confusion_matrix',cfm)
print(classification_report(Y_test,Y_pred))

In [ ]:
# Using Ensemble  model  with SVM  and  Random forest Classifiers
sns.heatmap(cfm,annot=True)
plt.show()